In [ ]:
#cnn
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
import os

EPOCH=1
BATCH_SIZE=50
LR=0.001
DOWNLOAD_MNIST=False

if not(os.path.exists('./mnist'))or not os.listdir('./mnist'):
    #不存在minist或者mnist是空的
    DOWNLOAD_MNIST=True

train_data=torchvision.datasets.MNIST(
    root='./mnist',
    train=True,#表示trainning data,6w个
    transform=torchvision.transforms.ToTensor(),#转换为tensor，并且像素点正则化到(0，1)
    download=DOWNLOAD_MNIST
    )
print(train_data.train_data.size())                 # (60000, 28, 28)
print(train_data.train_labels.size())               # (60000)
plt.imshow(train_data.train_data[0].numpy(), cmap='gray')
plt.title('%i' % train_data.train_labels[0])
plt.show()

train_loader=Data.DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
#每个mini-batch，image batch的shape为（50，1，28，28）

test_data=torchvision.datasets.MNIST(root='./mnist',train=False)
test_x=Variable(torch.unsqueeze(test_data.test_data,dim=1).type(torch.FloatTensor),volatile=True)[:2000]/255
 # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y=test_data.test_labels[:2000]
print(test_y[:12])

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(
                in_channels=1,#（1，28，28）
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2 #padding=(kernel_size-1)/2 if stride=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)#（16，14，14）
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(16,32,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(2)#（32，7，7）
        )
        self.out=nn.Linear(32*7*7,10)#全连接层, 10个分类
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)#（batch,32，7，7)
        x=x.view(x.size(0),-1)#size(0)保留batch,-1改编为32*7*7 #（batch,32*7*7)
        output=self.out(x)#32*7*7的输入，10的输出
        return output #x为了可视化，否则只output即可

cnn=CNN()
print(cnn)
print(cnn(test_x))
print(cnn(test_x).size())

optimizer=torch.optim.Adam(cnn.parameters(),lr=LR)
loss_func=nn.CrossEntropyLoss()



for epoch in range(EPOCH):
    for step ,(b_x,b_y)in enumerate(train_loader):
        b_x,b_y=Variable(b_x),Variable(b_y)
        output=cnn(b_x)
        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step%50==0:#每50步打印一次
            test_output=cnn(test_x)
            pred_y=torch.max(test_output,1)[1].data.numpy()
            accuracy=float((pred_y==test_y.data.numpy()).astype(int).sum())/float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
           

        
test_output= cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')
